Global setup


In [60]:
n=4
k=2
q=41
eta=2
R.<x> = ZZ[]

Bob's setup

In [61]:
Ax = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
Ax  #public

[ 40*x^3 + 34*x^2 + 23*x + 2    x^3 + 34*x^2 + 39*x + 34]
[38*x^3 + 13*x^2 + 28*x + 31  14*x^3 + 32*x^2 + 24*x + 8]

In [62]:
def binomial_centered():
    a1=randrange(0,2)
    a2=randrange(0,2)
    b1=randrange(0,2)
    b2=randrange(0,2)
#    return a1+a2-b1-b2
    return a1-b1
    return 0

In [63]:
sx = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
sx  #private

[x^3 + x^2]
[ -x^3 - x]

In [64]:
ex = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
ex #private

[          -x]
[-x^3 - x + 1]

In [65]:
tx = matrix(((Ax*sx + ex) % (x^n+1) % q))
tx  #public

[ 39*x^3 + 6*x^2 + 7*x - 17]
[18*x^3 - 17*x^2 - 28*x - 2]

**Matrices for lattice attack**


In [66]:
def negacirculant(v):
    Nshift = matrix([[0, 0, 0, -1],             #Negacyclic shift matrix
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]])
    return sum([v[i]*Nshift^i for i in range(0, len(v))])

In [67]:
Ax

[ 40*x^3 + 34*x^2 + 23*x + 2    x^3 + 34*x^2 + 39*x + 34]
[38*x^3 + 13*x^2 + 28*x + 31  14*x^3 + 32*x^2 + 24*x + 8]

In [68]:
A = block_matrix([[negacirculant(Ax[0,0].padded_list(4)), negacirculant(Ax[0,1].padded_list(4))], 
                  [negacirculant(Ax[1,0].padded_list(4)), negacirculant(Ax[1,1].padded_list(4))]])
A

[  2 -40 -34 -23| 34  -1 -34 -39]
[ 23   2 -40 -34| 39  34  -1 -34]
[ 34  23   2 -40| 34  39  34  -1]
[ 40  34  23   2|  1  34  39  34]
[---------------+---------------]
[ 31 -38 -13 -28|  8 -14 -32 -24]
[ 28  31 -38 -13| 24   8 -14 -32]
[ 13  28  31 -38| 32  24   8 -14]
[ 38  13  28  31| 14  32  24   8]

In [69]:
tx

[ 39*x^3 + 6*x^2 + 7*x - 17]
[18*x^3 - 17*x^2 - 28*x - 2]

In [70]:
t = block_matrix([[column_matrix(1, 4, tx[0,0].padded_list(4))],
                [column_matrix(1, 4, tx[1,0].padded_list(4))]])
t

[-17]
[  7]
[  6]
[ 39]
[---]
[ -2]
[-28]
[-17]
[ 18]

In [71]:
I8 = identity_matrix(8)
I8

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

**Eve's attack**

Eve knows $A$ so she can set up $M$:

In [72]:
M = block_matrix([[A, I8, -t, q*I8]])
M

[  2 -40 -34 -23  34  -1 -34 -39|  1   0   0   0   0   0   0   0| 17| 41   0   0   0   0   0   0   0]
[ 23   2 -40 -34  39  34  -1 -34|  0   1   0   0   0   0   0   0| -7|  0  41   0   0   0   0   0   0]
[ 34  23   2 -40  34  39  34  -1|  0   0   1   0   0   0   0   0| -6|  0   0  41   0   0   0   0   0]
[ 40  34  23   2   1  34  39  34|  0   0   0   1   0   0   0   0|-39|  0   0   0  41   0   0   0   0]
[ 31 -38 -13 -28   8 -14 -32 -24|  0   0   0   0   1   0   0   0|  2|  0   0   0   0  41   0   0   0]
[ 28  31 -38 -13  24   8 -14 -32|  0   0   0   0   0   1   0   0| 28|  0   0   0   0   0  41   0   0]
[ 13  28  31 -38  32  24   8 -14|  0   0   0   0   0   0   1   0| 17|  0   0   0   0   0   0  41   0]
[ 38  13  28  31  14  32  24   8|  0   0   0   0   0   0   0   1|-18|  0   0   0   0   0   0   0  41]

In [73]:
M.hermite_form()

[            1             0             0             0             1             0             1   47253723020    2672014319   -1418799032    -561637351    2868354327    -905235615     216648662      -6170328   -1217731533  -27070194676  109552587079  -58170760312  -23027131391  117602527407  -37114660215    8882595142    -252983448  -49926992853]
[            0             1             0             0             0             1             2  183643487123   10384325209   -5513919012   -2182707204   11147366964   -3518042905     841967854     -23979921   -4732504675 -105203667140  425757333569 -226070679492  -89490995364  457042045524 -144239759105   34520682014    -983176761 -194032691675]
[            0             0             1             0             1             0             1   70242356522    3971932166   -2109035671    -834870324    4263790330   -1345626942     322046848      -9172153   -1810150122  -40239670952  162849218806  -86470462511  -34229683284  174815403530  

In [74]:
A, U = M.transpose().hermite_form(transformation = true)
A.str(), U.str()

('[1 0 0 0 0 0 0 0]\n[0 1 0 0 0 0 0 0]\n[0 0 1 0 0 0 0 0]\n[0 0 0 1 0 0 0 0]\n[0 0 0 0 1 0 0 0]\n[0 0 0 0 0 1 0 0]\n[0 0 0 0 0 0 1 0]\n[0 0 0 0 0 0 0 1]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]',
 '[                        0                         0                         0                         0                         0                         0                         0                         0                         1                         0                         0                         0                         0                         0                         0                         0                         0                         0                         0                         0

In [75]:
M.right_kernel()

Free module of degree 25 and rank 17 over Integer Ring
Echelon basis matrix:
[  1   0   0   0   0   0   0   0   0  22  28  35   3  38  30  25  24 -10   3   2  21  -2 -18 -11   9]
[  0   1   0   0   0   0   0   0   0   0   8  24  14   2  14  39  12  -4   2   1  10   0  -9  -6   4]
[  0   0   1   0   0   0   0   0   0  13  10  14   9  23  17   8   2   0   1   0   1   0  -1  -2   0]
[  0   0   0   1   0   0   0   0   0  29  24  17   6  33  15   3  11  -4   2   2  10   0  -8  -4   4]
[  0   0   0   0   1   0   0   0   0  29  36   3  37  32   2  32  39 -17   5   4  37  -3 -28 -17  16]
[  0   0   0   0   0   1   0   0   0   5  12  31  38   0  16  39  29 -12   4   3  26  -2 -20 -13  11]
[  0   0   0   0   0   0   1   0   0  15  19  39  28  40  40  12   2   0   0  -1   0   0  -2  -2   0]
[  0   0   0   0   0   0   0   1   0  38  22   0  17  16  16  14  24  -9   4   3  22  -1 -16 -10  10]
[  0   0   0   0   0   0   0   0   1   2  31  17  17  33   1  11  12  -5   2   1  11  -1  -9  -5   5]
[  0 

In [76]:
M.right_kernel_matrix(basis='LLL')

[ 0  0 -1 -1  0  1  0  1  0  1  0  0 -1  1  0  1 -1  0 -2 -2 -2  0  0  0  0]
[-2  0  1 -1  0 -1  1 -3 -1 -2  4  0  1 -4 -1 -2 -4  0 -1  0  0  0  3  0  1]
[-4  0  1  0  0  5 -2 -1 -2  1  0 -1 -2 -4  2  2 -1 -1 -2  0  1  3  2 -2  0]
[ 0  0  0 -2 -1 -2  3 -1  1 -6 -1 -6  0 -2  0 -1 -2  2  0 -2 -2  0  2  0  1]
[-2  2  2  0  2  4 -6  3  1  4  1  1  2  2  2 -1  0  0  0  0 -1  2  0 -4 -1]
[-1 -2  1  0  1 -1  5 -1 -6 -1  0 -4 -2 -1  0  1  1  1  1 -2  0  2  3 -1 -1]
[ 1 -2  1  0  2  0 -1  1  2 -3 -5  2 -1 -1  1  2  3 -4  0  0  3 -3 -1 -2  0]
[ 1  0  0 -1  4 -3  0  4 -1 -2  5  2  0  0 -3 -4  2 -1  1 -2  0 -1 -1 -2  1]
[ 0  0 -1  0  0 -1  3 -3  1 -3  0  7  3  0 -1 -1 -3  0 -3 -2 -2  0  0  1 -1]
[ 0  0 -1  0  1 -4  1  0 -4  3 -4  0  0 -1 -2  0 -2  0  1  2  1 -1  1  3  2]
[-2 -3 -1  3 -1  3  4 -3 -1  1  2  1  0  0  1 -4  1 -1 -1  1  2  3  2  3 -2]
[-3 -2  2  2  4  4 -2  1  2  2  1  1  1  1 -3  1  0 -3 -1  0  1  2  3 -2 -3]
[ 0 -1  2  0  2 -2  3 -2 -1  0  1  7 -6 -1 -3  1 -1  0  0 -2 -1  0  2 -2 -2]

In [77]:
s = block_matrix([[column_matrix(1, 4, sx[0,0].padded_list(4))],
                [column_matrix(1, 4, sx[1,0].padded_list(4))]])
s

[ 0]
[ 0]
[ 1]
[ 1]
[--]
[ 0]
[-1]
[ 0]
[-1]

In [78]:
e = block_matrix([[column_matrix(1, 4, ex[0,0].padded_list(4))],
                [column_matrix(1, 4, ex[1,0].padded_list(4))]])
e

[ 0]
[-1]
[ 0]
[ 0]
[--]
[ 1]
[-1]
[ 0]
[-1]

In [79]:
matrix([1])

[1]

In [80]:
block_matrix([[s], [e], [matrix([1])], [matrix([[0],[0],[0],[0],[0],[0],[0],[0]])]])
_.str()

'[1]'

In [89]:
M*block_matrix([[s], [e], [matrix([1])], [matrix([[0],[2],[2],[2],[0],[0],[0],[0]])]])

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]

In [82]:
B, Transform = M.LLL(transformation=true)
B

[ 19  -8  29  13  -4  -1   6  29   1  -1   0   1   0   0   0   0 -15  41 -41   0  41   0   0   0   0]
[  8 -29 -13  19   1  -6 -29  -4   1   0  -1   1   0   0   0   0 -16  41   0 -41  41   0   0   0   0]
[ 29   2  21  -5 -26 -13   2  15  -1   0   0   0   1   0   0   0 -15 -41   0   0   0  41   0   0   0]
[ 31 -38 -13 -28   8 -14 -32 -24   0   0   0   0   1   0   0   0   2   0   0   0   0  41   0   0   0]
[ 42  -6 -11 -21  35  33   5  -5   1   0   0   1   0   0   0   0 -22  41   0   0  41   0   0   0   0]
[ -2 -21   5  29  13  -2 -15 -26   0   0   0  -1   0   0   0   1  21   0   0   0 -41   0   0   0  41]
[-21   5  29   2  -2 -15 -26 -13   0   0  -1   0   0   0   1   0  23   0   0 -41   0   0   0  41   0]
[ 28  31 -38 -13  24   8 -14 -32   0   0   0   0   0   1   0   0  28   0   0   0   0   0  41   0   0]

In [83]:
Transform

[ 1 -1  0  1  0  0  0  0]
[ 1  0 -1  1  0  0  0  0]
[-1  0  0  0  1  0  0  0]
[ 0  0  0  0  1  0  0  0]
[ 1  0  0  1  0  0  0  0]
[ 0  0  0 -1  0  0  0  1]
[ 0  0 -1  0  0  0  1  0]
[ 0  0  0  0  0  1  0  0]

In [84]:
Transform.determinant()

-1

In [85]:
Transform*M == B

True

Solve $M\mathbf{x} = \mathbf{t}$

In [86]:
xvector = B.solve_left(t.transpose())
xvector

ValueError: number of columns of self must equal number of columns of right-hand side

In [ ]:
xhat = xvector.apply_map(round, R=ZZ)
xhat

In [ ]:
xhat*Transform % q

In [ ]:
that = xhat*B
that

In [ ]:
t.transpose()-that

Solve $B\hat{\mathbf{s}} = \hat{\mathbf{t}}$

In [ ]:
shat = M.solve_left(that, check=true)
shat

In [ ]:
shat % q

In [ ]:
sx

In [ ]:
s-shat.transpose()[0:8]

In [ ]:
s = matrix([[1], [1], [1], [0],[-1], [1], [0], [1]])
s

**Alternative:  embedding**  (Knospe, p. 280)

In [ ]:
AZ = A.augment(q*I8)
AZ

In [ ]:
AZ.transpose().LLL().transpose()

In [ ]:
H = A.transpose().hermite_form(include_zero_rows=false).transpose()
H

In [ ]:
B = ((H.augment(t))).stack(vector([0,0,0,0,0,0,0,0,1])) % q
B

In [ ]:
B.transpose().LLL().transpose()

In [ ]:
e = block_matrix([[column_matrix(1, 4, ex[0,0].padded_list(4))],
                [column_matrix(1, 4, ex[1,0].padded_list(4))]])
e

Still doesn't work.  Is the error vector too large?  (Is any reasonable error vector too large?)